In [5]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import os 
import numpy as np 
import ast
import re 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle



In [11]:
df1=pd.read_csv('Cleaned beer data/Beerdata.csv')
df2=pd.read_csv('beer data/beer_data_step1.csv').set_index('beer_id')
df = df1.join(other=df2,on='beer_id',how='left')
df=df.rename(columns={'style_name':'Beer styles'})




df.reviews=df.reviews.apply(lambda x: ast.literal_eval(x))
df['beer_page']='https://www.beeradvocate.com'+df['beer_page']



df['N_rewiews']=df.reviews.str.len()
df_sample=df.loc[df.N_rewiews>0]
df_sample=df_sample.explode('reviews')


In [7]:
model3 = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3',device='cuda')

with open('embeddings_reviews3.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [17]:
def testing(my_sentence=''):
    sentence_embedding2 = model3.encode(my_sentence)

    cs=cosine_similarity(
        [sentence_embedding2],
        stored_embeddings,
        dense_output=True
    )
    #df_i=df_sample.iloc[cs[0].argmax()]
    df_=df_sample.iloc[cs[0].argsort()[::-1][0:5]]
    for i in range(len(df_)):
        df_i=df_.iloc[i]
        print(
            F'Review: {df_i.reviews}\n Style: {df_i.beer_style}\n beer name: {df_i.beer_name}\n beer page: {df_i.beer_page}\n\n\n'

            )

In [18]:
testing(my_sentence='good with burgers')

Review: Very good on a hot day or with a burger.
 Style: Helles
 beer name: Hell
 beer page: https://www.beeradvocate.com/beer/profile/5855/62170/



Review: Good beer, goes great with a burger and fries.
 Style: Märzen
 beer name: Oktoberfest
 beer page: https://www.beeradvocate.com/beer/profile/710/5757/



Review: This one was in the middle of the road for me. Personally was hoping for more flavor. This was somewhere between a blonde ale and a light beer. Not too bad though. Definitely went good with a burger.
 Style: American Blonde Ale
 beer name: Bloomin’ Blonde
 beer page: https://www.beeradvocate.com/beer/profile/26932/349578/



Review: Good beer with a burger and fries. It has a hoppy bitter taste, but not overpowering. It’s. It an “oh wow” beer but a solid choice. Not a watered down taste For sure.
 Style: American Pale Ale
 beer name: Manny's Pale Ale
 beer page: https://www.beeradvocate.com/beer/profile/4378/12645/



Review: Good solid beer that goes great with their fres